# <center>Big Data &ndash; Exercises &ndash; Solution</center>
## <center>Spring 2022 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to `exercise09` repository

2. Start docker <br>
```docker-compose up -d```

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: https://cloud.inf.ethz.ch/s/a8FoHew6dHKGYKK/download/confusion20140302.tbz2 

More specifically do the following:
- download the data      :<br> ```wget https://cloud.inf.ethz.ch/s/a8FoHew6dHKGYKK/download/confusion20140302.tbz2```
- extract the data       :<br> ```tar -jxvf confusion20140302.tbz2```

4. copy the data to hdfs :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to hdfs needs to be done only once and it might take 4-5 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: https://quietlyamused.org/blog/2014/03/12/language-confusion/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [38]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

#### <b>For the assignments we only use the first 50k lines of the dataset, `dataset_50k = dataset.limit(50000)`. </b>

## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a couple minutes).

### 1.0. Data preprocessing

In [39]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()

In [40]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



In [41]:
dataset_50k = dataset.limit(50000)

## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [42]:
start_exercise()

In [43]:
dataset_50k.filter("target = 'Russian' and target = guess").count()

957

In [44]:
finish_exercise()

This exercise took 1s


## Assignment 2
Return the number of distinct "target" languages.

In [45]:
start_exercise()

In [46]:
dataset_50k.select("target").distinct().count()

68

In [47]:
finish_exercise()

This exercise took 1s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [48]:
start_exercise()

In [49]:
dataset_50k.select("sample").filter("guess = target").orderBy(
    dataset_50k["target"].asc(), dataset_50k["country"].asc(), dataset_50k["date"].asc()
).limit(3).collect()

[Row(sample='fdf23d0a7063ba2fcef4b18eb7d57ad8'),
 Row(sample='13722ceed1eede7ba597ade9b4cb9807'),
 Row(sample='efcd813daec1c836d9f030b30caa07ce')]

In [50]:
finish_exercise()

This exercise took 2s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [51]:
start_exercise()

In [52]:
from pyspark.sql.functions import desc

dataset_50k.groupBy(["country", "target"]).count().orderBy(desc("count")).select("count").limit(3).collect()

[Row(count=770), Row(count=719), Row(count=715)]

In [53]:
finish_exercise()

This exercise took 1s


## Assignment 5
Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [54]:
start_exercise()

In [55]:
dataset_50k.filter("choices[0] = target and target = guess").count() / float(dataset_50k.count())

0.24352

In [56]:
finish_exercise()

This exercise took 6s


## Assignment 6
Return the number of games played on the latest day.

In [57]:
start_exercise()

In [58]:
max_date = dataset_50k.agg({"date": "max"}).collect()[0]["max(date)"]
dataset_50k.filter(dataset_50k["date"] == max_date).count()

34429

In [59]:
finish_exercise()

This exercise took 3s


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [ ]:
!pip install sparksql-magic

In [61]:
%load_ext sparksql_magic

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [62]:
path = "confusion-2014-03-02.json"
dataset_50k = spark.read.json(path).cache().limit(50000)
dataset_50k.registerTempTable("dataset_50k")

In [63]:
%%sparksql
-- test it out
SELECT *
FROM dataset_50k
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [64]:
start_exercise()

In [65]:
%%sparksql
-- write you query here
SELECT count(*) FROM dataset_50k
WHERE target == "Russian" 
AND target == guess

count(1)
957


In [66]:
finish_exercise()

This exercise took 2s


## Assignment 2
Return the number of distinct "target" languages.

In [67]:
start_exercise()

In [68]:
%%sparksql
-- write you query here
SELECT count(distinct(target))
FROM dataset_50k

count(DISTINCT target)
68


In [69]:
finish_exercise()

This exercise took 1s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [70]:
start_exercise()

In [71]:
%%sparksql
SELECT sample
FROM dataset_50k as d
WHERE guess = target
ORDER BY target asc, country asc, d.date asc
LIMIT 3

sample
fdf23d0a7063ba2fcef4b18eb7d57ad8
13722ceed1eede7ba597ade9b4cb9807
efcd813daec1c836d9f030b30caa07ce


In [72]:
finish_exercise()

This exercise took 3s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [73]:
start_exercise()

In [74]:
%%sparksql
SELECT count(guess)
FROM dataset_50k 
GROUP BY country, target
ORDER BY count(guess) desc
LIMIT 3

count(guess)
770
719
715


In [75]:
finish_exercise()

This exercise took 1s


## Assignment 5
Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [76]:
start_exercise()

In [77]:
%%sparksql
SELECT count(*) / 
    (SELECT count(*) as count_all
    FROM dataset_50k)
FROM dataset_50k
WHERE choices[0] = target AND target = guess

(CAST(count(1) AS DOUBLE) / CAST(scalarsubquery() AS DOUBLE))
0.24352


In [78]:
finish_exercise()

This exercise took 2s


## Assignment 6
Return the number of games played on the latest day.

In [79]:
start_exercise()

In [80]:
%%sparksql
SELECT count(*) 
FROM dataset_50k
WHERE date = 
(
    SELECT max(date)
    FROM dataset_50k
)

count(1)
34429


In [81]:
finish_exercise()

This exercise took 1s
